# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

- **Consider a significance level of 5% for all tests.**

In [1]:
# import numpy and pandas
import pandas as pd
import numpy as np
import scipy.stats as st

# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [2]:
# Run this code:

pokemon = pd.read_csv('../pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [3]:
# Your code here:

pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [4]:
# Your code here:

regular_poke = pokemon[pokemon['Legendary'] == False]
legendary_poke = pokemon[pokemon['Legendary'] == True]

print(regular_poke['Legendary'].count())
print(legendary_poke['Legendary'].count())

735
65


Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [5]:
# Your code here:

print(f"Mean of regular pokes is: {regular_poke['Total'].mean()} and STD is {regular_poke['Total'].std()}")
print(f"Mean of legendary pokes is: {legendary_poke['Total'].mean()} and STD is {legendary_poke['Total'].std()}")

Mean of regular pokes is: 417.21360544217686 and STD is 106.76041745713022
Mean of legendary pokes is: 637.3846153846154 and STD is 60.93738905315346


The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [6]:
#1. Hypotesis
    #H0: mu legendary pokemons <= mu regular pokemons
    #H1: mu legendary pokemons > mu regular pokemons

#2. Samples
regular_poke['Total']
legendary_poke['Total']


#3. Confidence --> 95%
a=0.05

#4&5 
stat, p_value = st.ttest_ind(regular_poke['Total'],legendary_poke['Total'],equal_var=False)
print(f'Stat is {stat} and p_value is {p_value}') 

Stat is -25.8335743895517 and p_value is 9.357954335957446e-47


What do you conclude from this test? Write your conclusions below.

In [7]:
# Your conclusions here:
#6.Decision
p_value < a  
## Rejected H0 --> rejecting that legendary pokemons have a lesser mean value than regular pokemons

True

How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [24]:
# Your code here:

pokemon['Type 1'].value_counts().sort_values()

Flying        4
Fairy        17
Ice          24
Fighting     27
Steel        27
Poison       28
Dark         31
Ghost        32
Ground       32
Dragon       32
Electric     44
Rock         44
Fire         52
Psychic      57
Bug          69
Grass        70
Normal       98
Water       112
Name: Type 1, dtype: int64

Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [33]:
# Your code here:
water_pokes =  pokemon[pokemon['Type 1'] == 'Water']
other_types_pokes =  pokemon[pokemon['Type 1'] != 'Water']


print(f"Mean of water pokes is: {water_pokes['Total'].mean()} and STD is {water_pokes['Total'].std()}")
print(f"Mean of NON-water pokes is: {other_types_pokes['Total'].mean()} and STD is {other_types_pokes['Total'].std()}")



Mean of water pokes is: 430.45535714285717 and STD is 113.18826606431458
Mean of NON-water pokes is: 435.85901162790697 and STD is 121.09168230208066


Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [34]:
#1. Hypotesis
## I suspect that the mu of the water pokemons is equal to all other pokemons
    #H0: mu water pokemons = mu non-water pokemons
    #H1: mu water pokemons != mu non-water pokemons

#2. Samples
water_pokes['Total']
other_types_pokes['Total']


#3. Confidence --> 95%
a=0.05

#4&5 
stat, p_value = st.ttest_ind(water_pokes['Total'],other_types_pokes['Total'],equal_var=True)
print(f'Stat is {stat} and p_value is {p_value}') 


Stat is -0.4418547448849676 and p_value is 0.6587140317488793


Write your conclusion below.

In [36]:
#6. Decision
p_value < a
# Since p_value in bigger than alpha I DO NOT have enough evidence to reject the null hypotesis that 
# the mean of water pokes is equal to the mean of the other pokes and so I continue to believe that they are equal

False

# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [40]:
pokemon

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


In [51]:
#1. Hypotesis
    #H0: mu special defense = mu special attack
    #H1: mu special defense != mu special attack

#2. Samples
print(pokemon['Attack'].mean())
print(pokemon['Defense'].mean())

#3. Confidence --> 95%
a=0.05

#4&5 
stat, p_value = st.ttest_rel(pokemon['Attack'],pokemon['Defense'])
print(f'Stat is {stat} and p_value is {p_value}') 

79.00125
73.8425
Stat is 4.325566393330478 and p_value is 1.7140303479358558e-05


Describe the results of the test in the cell below.

In [52]:
#6. Decision
p_value < a
# Since p_value in bigger than alpha I have enough evidence to reject the null hypotesis that 
# the mean of attack is equal to the mean of the defense


True

We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [53]:
#1. Hypotesis
    #H0: mu defense = mu attack
    #H1: mu defense != mu attack

#2. Samples
print(pokemon['Sp. Atk'].mean())
print(pokemon['Sp. Def'].mean())

#3. Confidence --> 95%
a=0.05

#4&5 
stat, p_value = st.ttest_rel(pokemon['Sp. Atk'],pokemon['Sp. Def'])
print(f'Stat is {stat} and p_value is {p_value}') 

72.82
71.9025
Stat is 0.853986188453353 and p_value is 0.3933685997548122


Describe the results of the test in the cell below.

In [55]:
#6. Decision
p_value < a
# Since p_value in bigger than alpha I DO NOT have enough evidence to reject the null hypotesis that 
# the mean of special attack is equal to the mean of the special defense

False

As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [61]:
#1. Hypotesis
    #H0: defense - attack = 0
    #H1: defense - attack != 0

#2. Samples
atk_def_diff = pokemon['Defense'] - pokemon['Attack']

#3. Confidence --> 95%
a=0.05

#4&5 
stat, p_value = st.ttest_1samp(atk_def_diff,0)
print(f'Stat is {stat} and p_value is {p_value}') 

#6. Decision
p_value < a
# Since p_value in smaller than alpha I have enough evidence to reject the null hypotesis that 
# the diff defense and attack is equal to 0

Stat is -4.325566393330478 and p_value is 1.7140303479358558e-05


True

# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [66]:
# Your code here:
pokemon['Water_pokes'] = pokemon['Type 1'] == 'Water'

ct = pd.crosstab(index=pokemon['Water_pokes'], columns=pokemon['Legendary'])
ct

Legendary,False,True
Water_pokes,,
False,627,61
True,108,4


Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [68]:
### H0: The poke being legendary is independent from being water
### H1: The poke being legendary dependent from being water

# significance level
a=0.05

##Sample
ct

### 4. and 5 Stats and p_value
statistic_all = st.chi2_contingency(ct)
print(statistic_all)
### Tuple from chi2_contigency:
    ##1st: stat
    ##2nd: p_value
    ##3rd: degrees of freedom
    ##4th: expected value if H0 stands true (if the cars are independent from perception)

(2.9429200762850503, 0.08625467249550949, 1, array([[632.1,  55.9],
       [102.9,   9.1]]))


Based on a 95% confidence, should we reject the null hypothesis?

In [69]:
##Decision
statistic_all[1] < a ## If False we do not reject, if True we reject the hypotesis with a confidence of 95% --> 
                     ## So we gathered enough information to continue believing that the 2 are independent

False